DIRECTORIO

In [1]:
#cd '/tf/PT_JoseVeloso/Mask_RCNN-master/'

In [2]:
pwd

'/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training'

In [3]:
cd ..

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [4]:
ls

LICENSE                   damage_cfg20220812T0733/  logs/
MANIFEST.in               damage_cfg20220812T0734/  mask_rcnn.egg-info/
README.md                 damage_cfg20220812T0738/  mask_rcnn_balloon.h5
app/                      damage_cfg20220812T0739/  mask_rcnn_coco.h5
assets/                   damage_cfg20220812T0819/  model/
build/                    damage_cfg20220817T0537/  model-training/
customImages/             damage_cfg20220817T0538/  mrcnn/
damage_cfg20220802T0650/  damage_cfg20220817T0541/  report.txt
damage_cfg20220802T1832/  damage_cfg20220817T0622/  requirements.txt
damage_cfg20220804T0653/  datasets/                 samples/
damage_cfg20220804T0658/  dist/                     setup.cfg
damage_cfg20220812T0652/  images/                   setup.py
damage_cfg20220812T0654/  instalacion


# Import Libraries

In [5]:
# import basic libraries
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# import advance libraries
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# import mask rcnn libraries
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# import matplotlib library
import matplotlib.pyplot as plt

# import numpy libraries
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# import keras libraries
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [6]:
%matplotlib inline

# Stage-1 Training (Single Class & Bounding Box Annotations)

In Stage-1 training I used a simple dataset with images annotated using Bounding Boxes, and one class namely 'Damage'. In the following section you can find the code for training of the model. I have included comments to best desccribe the flow of the program.

In [9]:
class DamageDataset(Dataset):
    
    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, dataset_dir, is_train=True):
        
        # we add a class that we need to classify in our case it is Damage
        
        self.add_class("dataset", 1, "Damage")
        #self.add_class("dataset", 1, "arandano")
        
        # we concatenate the dataset_dir with /images and /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train will be true if we our training our model and false when we are testing the model
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            
            # if is_train is True skip all images with id greater than and equal to 420
            # roughly 80% of dataset for training
            #frame0985
            
            #if is_train and int(image_id) >= 3835 :
            if is_train and int(image_id) >= 420 :
                print("image_id: ", image_id)
                continue
            
            # if is_train is not True skip all images with id less than 420
            #if not is_train and int(image_id) < 3835:
            if not is_train and int(image_id) < 420:
                continue
            
            # declaring image path and annotations path
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training or testing
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        # you can see how the images are annotated we extracrt the width, height and bndbox values
        
        # <annotation>
        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>
        # </annotation>
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in image
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # we append the class_id 1 for Damage in our case to the variable
        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example:  if 2.jpg have three objects we will have following masks and class_ids
        # 000000000 000000000 000001110 
        # 000011100 011100000 000001110
        # 000011100 011100000 000001110
        # 000000000 011100000 000000000
        #    1         1          1    <- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('Damage'))
            #class_ids.append(self.class_names.index('arandano'))
        
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

# damage configuration class, you can change values of hyper parameters here
class DamageConfig(Config):
    # name of the configuration
    NAME = "damage_cfg"
    
    # damage class + background class
    NUM_CLASSES = 1 + 1
    
    # steps per epoch and minimum confidence
    #STEPS_PER_EPOCH = 361
    STEPS_PER_EPOCH = 180
    
    # learning rate and momentum
    LEARNING_RATE=0.002
    LEARNING_MOMENTUM = 0.8
    
    # regularization penalty
    WEIGHT_DECAY = 0.0001
    
    # image size is controlled by this parameter
    IMAGE_MIN_DIM = 512
    
    # validation steps
    #VALIDATION_STEPS = 50
    VALIDATION_STEPS = 50
    
    # number of Region of Interest generated per image
    #Train_ROIs_Per_Image = 200
    Train_ROIs_Per_Image = 100
    
    # RPN Acnhor scales and ratios to find ROI
    #RPN_ANCHOR_SCALES = (16, 32, 48, 64, 128)
    RPN_ANCHOR_SCALES = (16, 32, 48, 64, 128)
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]

# load the train dataset
train_set = DamageDataset()
train_set.load_dataset('customImages/stage-1', is_train=True)
#train_set.load_dataset('customImages/IMG_train', is_train=True)
train_set.prepare()

# load the test dataset
test_set = DamageDataset()
test_set.load_dataset('customImages/stage-1', is_train=False)
#test_set.load_dataset('customImages/IMG_train', is_train=False)
test_set.prepare()

# prepare config by calling the user defined confifuration class
config = DamageConfig()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)

# load weights mscoco model weights
weights_path = 'mask_rcnn_coco.h5'

# load the model weights
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')

#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=3, layers='heads')

image_id:  522
image_id:  472
image_id:  556
image_id:  461
image_id:  442
image_id:  499
image_id:  514
image_id:  505
image_id:  421
image_id:  455
image_id:  587
image_id:  447
image_id:  504
image_id:  468
image_id:  434
image_id:  567
image_id:  561
image_id:  518
image_id:  596
image_id:  571
image_id:  560
image_id:  530
image_id:  562
image_id:  574
image_id:  515
image_id:  570
image_id:  594
image_id:  507
image_id:  493
image_id:  564
image_id:  529
image_id:  464
image_id:  536
image_id:  466
image_id:  575
image_id:  496
image_id:  444
image_id:  454
image_id:  478
image_id:  519
image_id:  533
image_id:  501
image_id:  438
image_id:  581
image_id:  448
image_id:  502
image_id:  547
image_id:  583
image_id:  446
image_id:  436
image_id:  535
image_id:  548
image_id:  549
image_id:  558
image_id:  550
image_id:  451
image_id:  477
image_id:  543
image_id:  420
image_id:  572
image_id:  528
image_id:  590
image_id:  526
image_id:  576
image_id:  490
image_id:  440
image_id: 

KeyboardInterrupt: 

# Stage-2 (Mulptiple Class and Bounding Box Annotaions)

In the stage I trained the model on multiple classes namely level-1 (scratch), level-2 (dent), level-3 (shatter), and level-4 (dislocation). The images are annotated using bounding boxes.

I recommend going through Stag-1 Training code first, there is not much difference between code of stage 1 and stage 2.

## Model Training

In [ ]:
class DamageDataset(Dataset):

    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, dataset_dir, is_train=True):
        
        # we use add_class for each class in our dataset and assign numbers to them. 0 is background
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "Level-1")
        self.add_class("dataset", 2, "Level-2")
        self.add_class("dataset", 3, "Level-3")
        self.add_class("dataset", 4, "Level-4")
        
        # we concatenate the dataset_dir with /images and /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train will be true if we our training our model and false when we are testing the model
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            # if is_train is True skip all images with id greater than and equal to 160
             # roughly 80% of dataset for training
            if is_train and int(image_id) >= 160 :
                continue
             
            # if is_train is not True skip all images with id less than 420
            if not is_train and int(image_id) < 160:
                continue
            
            # img_path and ann_path variables are defined
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    
    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        # you can see how the images are annotated we extracrt the width, height and bndbox values

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in an image
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
            
            # I have included this line to skip any un-annotated images
            if name=='Level-1' or name=='Level-2' or name=='Level-3' or name=='Level-4':
                boxes.append(coors)

        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances        
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example: if 2.jpg have four objects we will have following masks and class_ids
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'Level-1'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('Level-1'))
            elif(box[4] == 'Level-2'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('Level-2')) 
            elif(box[4] == 'Level-3'):
                masks[row_s:row_e, col_s:col_e, i] = 3
                class_ids.append(self.class_names.index('Level-3'))
            else:
                masks[row_s:row_e, col_s:col_e, i] = 4
                class_ids.append(self.class_names.index('Level-4'))
                
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

# damage configuration class, you can change values of hyper parameters here
class DamageConfig(Config):
    # name of the configuration
    NAME = "damage_cfg"
    
    #  background class + 4 classes
    NUM_CLASSES = 1 + 4
    
    # steps per epoch and minimum confidence
    STEPS_PER_EPOCH = 160
    
    # learning rate and momentum
    LEARNING_RATE=0.002
    LEARNING_MOMENTUM = 0.8
    
    # regularization penalty
    WEIGHT_DECAY = 0.0001
    
    # image size is controlled by this parameter
    IMAGE_MIN_DIM = 512
    
    # validation steps
    VALIDATION_STEPS = 50
    
    # number of Region of Interest generated per image
    Train_ROIs_Per_Image = 200
    
    # RPN Acnhor scales and ratios to find ROI
    RPN_ANCHOR_SCALES = (16, 32, 48, 64, 128)
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]

# prepare train set
train_set = DamageDataset()
train_set.load_dataset('customImages/stage-2', is_train=True)
train_set.prepare()

# validation/test
test_set = DamageDataset()
test_set.load_dataset('customImages/stage-2', is_train=False)
test_set.prepare()

# load damage config
config = DamageConfig()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)

# load weights mscoco model weights
weights_path = 'mask_rcnn_coco.h5'

# load the model weights
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])


# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, 
            test_set, 
            learning_rate=config.LEARNING_RATE, 
            epochs=5, 
            layers='heads')

# Stage 3 ( Multiple Classes and images annotated with polygon)

In this stage I have used multiple classes namely scratch, dent, shatter and dislocation. The images are annotated using a polygon. 

## Model Training

In [ ]:
class DamageDataset(Dataset):

    def load_dataset(self, dataset_dir, subset):
        
        # we use add_class for each class in our dataset and assign numbers to them. 0 is background
        # self.add_class('source', 'class id', 'class name')
        self.add_class("damage", 1, "Scratch")
        self.add_class("damage", 2, "Dent")
        self.add_class("damage", 3, "Shatter")
        self.add_class("damage", 4, "Dislocation")
        
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        
        # load annotations using json.load()
        annotations1 = json.load(open(os.path.join(dataset_dir, "via_project.json")))
        
        # convert annotations1 into a list
        annotations = list(annotations1.values())  
        
        # we only require the regions in the annotations
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            
            # extracting shape attributes and region attributes
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['damage'] for s in a['regions']]
            
            # create a dictionary {name_of_class: class_id} remember background has id 0
            name_dict = {"Scratch": 1, "Dent": 2, "Shatter": 3, "Dislocation": 4}
            
            # all the ids/classes in a image
            num_ids = [name_dict[a] for a in objects]
            
            # you can print these ids
            # print("numids",num_ids)
            
            # read image and get height and width
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            # add image to the dataset
            self.add_image(
                "damage",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )
            
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # for cases when source is not damage
        if info["source"] != "damage":
            return super(self.__class__, self).load_mask(image_id)
        
        # get the class ids in an image
        num_ids = info['num_ids']
        
        
        
        # we create len(info["polygons"])(total number of polygons) number of masks of height 'h' and width 'w'
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        
        # we loop over all polygons and generate masks (polygon mask) and class id for each instance
        # masks can have any shape as we have used polygon for annotations
        # for example: if 2.jpg have four objects we will have following masks and class_ids
        # 000001100 000111000 000001110
        # 000111100 011100000 000001110
        # 000011111 011111000 000001110
        # 000000000 111100000 000000000
        #    1         2          3    <- class_ids
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

            mask[rr, cc, i] = 1
            
        # return masks and class_ids as array
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "damage":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

# define a configuration for the model
class DamageConfig(Config):
    # define the name of the configuration
    NAME = "damage"
    
    # number of classes (background + damge classes)
    NUM_CLASSES = 1 + 4
    
    # number of training steps per epoch
    STEPS_PER_EPOCH = 160
    # learning rate and momentum
    LEARNING_RATE=0.002
    LEARNING_MOMENTUM = 0.8
    
    # regularization penalty
    WEIGHT_DECAY = 0.0001
    
    # image size is controlled by this parameter
    IMAGE_MIN_DIM = 512
    
    # validation steps
    VALIDATION_STEPS = 50
    
    # number of Region of Interest generated per image
    Train_ROIs_Per_Image = 200
    
    # RPN Acnhor scales and ratios to find ROI
    RPN_ANCHOR_SCALES = (16, 32, 48, 64, 128)
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]

# prepare train dataset.
train_set = DamageDataset()
# change the dataset 
train_set.load_dataset("customImages/stage-3", "train")
train_set.prepare()

# prepare validation/test dataset
test_set = DamageDataset()
test_set.load_dataset("customImages/stage-3", "val")
test_set.prepare()

# load damage config
config = DamageConfig()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)

# load weights mscoco model weights
weights_path = 'mask_rcnn_coco.h5'

# load the model weights
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, 
            test_set, 
            learning_rate=config.LEARNING_RATE, 
            epochs=15, 
            layers='heads')

# Model Evaluation (Stage-3)

In [ ]:
# we define a prediction configuration 
class PredictionConfig(Config):
    NAME = "damage"
    NUM_CLASSES = 1 + 4
    DETECTION_MIN_CONFIDENCE = 0.85
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


# evaluate_model is used to calculate mean Average Precision of the model
def evaluate_model(dataset, model, cfg):
    APs = list()
    for image_id in dataset.image_ids:

        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)

        scaled_image = mold_image(image, cfg)

        sample = expand_dims(scaled_image, 0)

        yhat = model.detect(sample, verbose=0)

        r = yhat[0]

        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])

        APs.append(AP)
    maP = mean(APs)
    return mAP

# train dataset
train_set = DamageDataset()
train_set.load_dataset("customImages/stage-3", "train")
train_set.prepare()

# Validation dataset
test_set = DamageDataset()
test_set.load_dataset("customImages/stage-3", "val")
test_set.prepare()

# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model.load_weights('your_trained_model_weights.h5', by_name=True)

# evaluate model on train dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP[0])

# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP[1])

# Single Image Prediction (Stage-3)

In [ ]:
# define a configuration for the model
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "damage"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 4
    # number of training steps per epoch
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
# load prediction configuration
cfg = PredictionConfig()

# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

# load model weights
model_path = 'your_trained_weights.h5'
model.load_weights(model_path, by_name=True)

In [ ]:
image = load_img("image_path.jpg")
image = img_to_array(image)

results = model.detect([image], verbose=1)

class_names = ['BG', 'Scratch', 'Dent', 'Shatter', 'Dislocation']

r = results[0]

visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names,  r['scores'])